In [27]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
#from train import create_dir, load_dataset

In [28]:
global image_h
global image_w
global num_landmarks

In [29]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
# create_dir("results")

""" Hyperparameters """
image_h = 224
image_w = 224
num_landmarks = 106

""" Paths """
path = "./model_result/result ini_lr_0.001"
model_path = os.path.join(path, "model_effi.h5")

In [30]:
# def plot_landmarks(image, landmarks):
#     h, w, _ = image.shape
#     radius = int(h * 0.005)

#     for i in range(0, len(landmarks), 2):
#         x = int(landmarks[i] * w)
#         y = int(landmarks[i+1] * h)

#         image = cv2.circle(image, (x, y), radius, (255, 0, 0), -1)

#     return image

In [31]:
def plot_landmarks2(image, landmarks):
    h, w, _ = image.shape
    
    for i in range(0, len(landmarks), 2):
        x = int(landmarks[i]*w)
        y = int(landmarks[i+1]*h)
        #print(x,y)
        image = cv2.circle(image,(x, y), 3, (0,255,0), -1)
        num = int(i/2 + 1)
        num_text = str(num)
        text_origin = (x,y+10)
        TEXT_FACE = cv2.FONT_HERSHEY_DUPLEX
        TEXT_SCALE = 0.3
        TEXT_THICKNESS = 1
        image = cv2.putText(image, num_text, text_origin, TEXT_FACE, TEXT_SCALE, (0,0,255), TEXT_THICKNESS, cv2.LINE_AA)
    
    return image

In [32]:
""" Load the model """
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  0           ['rescaling[0][0]']              
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['normalization[0][0]']      

 ivation)                                                                                         
                                                                                                  
 block3b_project_conv (Conv2D)  (None, 28, 28, 48)   9216        ['block3b_expand_activation[0][0]
                                                                 ']                               
                                                                                                  
 block3b_project_bn (BatchNorma  (None, 28, 28, 48)  192         ['block3b_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block3b_drop (Dropout)         (None, 28, 28, 48)   0           ['block3b_project_bn[0][0]']     
                                                                                                  
 block3b_a

 block4b_add (Add)              (None, 14, 14, 96)   0           ['block4b_drop[0][0]',           
                                                                  'block4a_project_bn[0][0]']     
                                                                                                  
 block4c_expand_conv (Conv2D)   (None, 14, 14, 384)  36864       ['block4b_add[0][0]']            
                                                                                                  
 block4c_expand_bn (BatchNormal  (None, 14, 14, 384)  1536       ['block4c_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block4c_expand_activation (Act  (None, 14, 14, 384)  0          ['block4c_expand_bn[0][0]']      
 ivation)                                                                                         
          

                                                                                                  
 block5b_dwconv2 (DepthwiseConv  (None, 14, 14, 672)  6048       ['block5b_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block5b_bn (BatchNormalization  (None, 14, 14, 672)  2688       ['block5b_dwconv2[0][0]']        
 )                                                                                                
                                                                                                  
 block5b_activation (Activation  (None, 14, 14, 672)  0          ['block5b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5b_s

 )                                                                                                
                                                                                                  
 block5d_activation (Activation  (None, 14, 14, 672)  0          ['block5d_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5d_se_squeeze (GlobalAver  (None, 672)         0           ['block5d_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5d_se_reshape (Reshape)   (None, 1, 1, 672)    0           ['block5d_se_squeeze[0][0]']     
                                                                                                  
 block5d_s

 block6a_se_squeeze (GlobalAver  (None, 672)         0           ['block6a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6a_se_reshape (Reshape)   (None, 1, 1, 672)    0           ['block6a_se_squeeze[0][0]']     
                                                                                                  
 block6a_se_reduce (Conv2D)     (None, 1, 1, 28)     18844       ['block6a_se_reshape[0][0]']     
                                                                                                  
 block6a_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block6a_se_reduce[0][0]']      
                                                                                                  
 block6a_se_excite (Multiply)   (None, 7, 7, 672)    0           ['block6a_activation[0][0]',     
          

                                                                  'block6c_se_expand[0][0]']      
                                                                                                  
 block6c_project_conv (Conv2D)  (None, 7, 7, 192)    221184      ['block6c_se_excite[0][0]']      
                                                                                                  
 block6c_project_bn (BatchNorma  (None, 7, 7, 192)   768         ['block6c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6c_drop (Dropout)         (None, 7, 7, 192)    0           ['block6c_project_bn[0][0]']     
                                                                                                  
 block6c_add (Add)              (None, 7, 7, 192)    0           ['block6c_drop[0][0]',           
          

 lization)                                                                                        
                                                                                                  
 block6e_drop (Dropout)         (None, 7, 7, 192)    0           ['block6e_project_bn[0][0]']     
                                                                                                  
 block6e_add (Add)              (None, 7, 7, 192)    0           ['block6e_drop[0][0]',           
                                                                  'block6d_add[0][0]']            
                                                                                                  
 block6f_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6e_add[0][0]']            
                                                                                                  
 block6f_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block6f_expand_conv[0][0]']    
 ization) 

                                                                  'block6f_add[0][0]']            
                                                                                                  
 block6h_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6g_add[0][0]']            
                                                                                                  
 block6h_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block6h_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6h_expand_activation (Act  (None, 7, 7, 1152)  0           ['block6h_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6h_d

In [33]:
x = './LaPa/test/onebin.jpg'
x

'./LaPa/test/onebin.jpg'

In [34]:
""" Reading the image """
image = cv2.imread(x, cv2.IMREAD_COLOR)
image_x = image

image = cv2.resize(image, (image_w, image_h))

image = image/255.0 ## (512, 512, 3)
image = np.expand_dims(image, axis=0) ## (1, 512, 512, 3)
image = image.astype(np.float32)
image

array([[[[0.05490196, 0.08627451, 0.11372549],
         [0.05882353, 0.09019608, 0.11764706],
         [0.05882353, 0.09019608, 0.11764706],
         ...,
         [0.39215687, 0.56078434, 0.6862745 ],
         [0.39215687, 0.5647059 , 0.6862745 ],
         [0.39607844, 0.5686275 , 0.6901961 ]],

        [[0.05490196, 0.08627451, 0.11372549],
         [0.05882353, 0.09019608, 0.11764706],
         [0.0627451 , 0.09411765, 0.12156863],
         ...,
         [0.39607844, 0.5647059 , 0.69411767],
         [0.39215687, 0.5647059 , 0.6862745 ],
         [0.39607844, 0.5686275 , 0.6901961 ]],

        [[0.0627451 , 0.09019608, 0.12156863],
         [0.06666667, 0.09803922, 0.1254902 ],
         [0.07450981, 0.10588235, 0.13333334],
         ...,
         [0.4       , 0.5686275 , 0.69803923],
         [0.39607844, 0.5686275 , 0.6901961 ],
         [0.39215687, 0.5647059 , 0.6901961 ]],

        ...,

        [[0.1764706 , 0.3137255 , 0.44313726],
         [0.18039216, 0.31764707, 0.4509804 ]

In [35]:
""" Prediction """
pred = model.predict(image, verbose=0)[0]
pred = pred.astype(np.float32)
pred

array([0.21069336, 0.4633789 , 0.20703125, 0.50341797, 0.2006836 ,
       0.5410156 , 0.20227051, 0.58154297, 0.20703125, 0.6171875 ,
       0.20605469, 0.66259766, 0.22070312, 0.7001953 , 0.22912598,
       0.73828125, 0.24108887, 0.7783203 , 0.2626953 , 0.7993164 ,
       0.29101562, 0.8256836 , 0.32495117, 0.85546875, 0.3540039 ,
       0.8769531 , 0.3930664 , 0.88916016, 0.44140625, 0.9003906 ,
       0.47827148, 0.90966797, 0.52001953, 0.91015625, 0.5600586 ,
       0.9003906 , 0.5913086 , 0.88720703, 0.6044922 , 0.8696289 ,
       0.6333008 , 0.84472656, 0.6435547 , 0.81591797, 0.6621094 ,
       0.7910156 , 0.6801758 , 0.765625  , 0.6948242 , 0.73046875,
       0.7036133 , 0.6958008 , 0.6933594 , 0.6557617 , 0.703125  ,
       0.6225586 , 0.69921875, 0.59521484, 0.7182617 , 0.5703125 ,
       0.7036133 , 0.51904297, 0.70751953, 0.48950195, 0.7104492 ,
       0.4663086 , 0.31176758, 0.4506836 , 0.36035156, 0.43652344,
       0.39160156, 0.4099121 , 0.45898438, 0.41064453, 0.50439

In [36]:
""" Saving the results """
#gt_landmarks = plot_landmarks(image_x.copy(), landmarks)
result_image = plot_landmarks2(image_x.copy(), pred)
# line = np.ones((image_x.shape[0], 10, 3)) * 255

In [37]:
print(type(result_image))

<class 'numpy.ndarray'>


In [38]:
result_image

array([[[ 15,  23,  30],
        [ 15,  23,  30],
        [ 15,  23,  30],
        ...,
        [100, 144, 175],
        [101, 145, 176],
        [101, 145, 176]],

       [[ 14,  22,  29],
        [ 14,  22,  29],
        [ 14,  22,  29],
        ...,
        [101, 145, 176],
        [101, 145, 176],
        [101, 145, 176]],

       [[ 14,  22,  29],
        [ 14,  22,  29],
        [ 14,  22,  29],
        ...,
        [101, 145, 176],
        [101, 145, 176],
        [101, 145, 176]],

       ...,

       [[ 45,  78, 111],
        [ 46,  79, 112],
        [ 45,  80, 113],
        ...,
        [125, 147, 165],
        [133, 155, 173],
        [130, 152, 170]],

       [[ 44,  77, 110],
        [ 46,  79, 112],
        [ 47,  80, 113],
        ...,
        [128, 150, 168],
        [125, 147, 165],
        [121, 143, 161]],

       [[ 44,  77, 110],
        [ 46,  79, 112],
        [ 47,  80, 113],
        ...,
        [128, 150, 168],
        [122, 144, 162],
        [119, 141, 159]]

In [39]:
# cat_image = np.concatenate([gt_landmarks, line, pred_landmarks], axis=1)
name = "onebin_effi_2-1"
cv2.imwrite(f"./result2/{name}.png", result_image)

True